# worker

> Generic worker process for executing plugin-based jobs in isolated subprocesses.

In [ ]:
#| default_exp core.worker

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import multiprocessing
import queue
from typing import Dict, Any, Optional, Callable

from cjm_fasthtml_workers.core.protocol import (
    WorkerRequestType,
    WorkerResponseType,
    PluginManagerAdapter,
)

## Worker Process

The `base_worker_process` function is the entry point for worker subprocesses. It:

1. Receives plugin configurations through an init message
2. Creates a plugin manager instance (isolated from the main process)
3. Loads configured plugins
4. Processes job execution requests from the request queue
5. Sends results back through the result queue
6. Handles plugin reload/unload commands
7. Supports streaming execution for incremental results

In [ ]:
#| export
def base_worker_process(
    request_queue: multiprocessing.Queue,  # Queue for receiving job requests from parent
    result_queue: multiprocessing.Queue,  # Queue for sending job results back to parent
    response_queue: multiprocessing.Queue,  # Queue for sending command responses back to parent
    plugin_manager_factory: Callable[[], PluginManagerAdapter],  # Factory function that creates a plugin manager instance
    result_adapter: Optional[Callable[[Any], Dict[str, Any]]] = None,  # Optional function to adapt plugin results to dict format
    supports_streaming: bool = False  # Whether this worker supports streaming execution
):
    """
    Generic long-lived worker process that handles job execution.
    
    The worker runs in an isolated subprocess, receiving jobs from the parent process
    via queues and sending results back. It manages plugin lifecycle and can be 
    terminated for cancellation.
    """
    try:
        plugin_manager: Optional[PluginManagerAdapter] = None
        plugin_configs: Dict[str, Dict[str, Any]] = {}

        # Track current worker state
        current_plugin: Optional[str] = None  # Currently loaded plugin name
        current_config: Optional[Dict[str, Any]] = None  # Current plugin config

        # Process jobs in a loop
        while True:
            try:
                # Wait for a request
                request = request_queue.get(timeout=1.0)
                request_type = request.get('type')

                if request_type == WorkerRequestType.STOP.value:
                    # Graceful shutdown
                    break

                elif request_type == WorkerRequestType.INIT.value:
                    # Initialize plugin manager with configurations from parent
                    plugin_configs = request.get('plugin_configs', {})

                    # Create plugin manager using factory
                    plugin_manager = plugin_manager_factory()

                    # Discover plugins in worker subprocess
                    discovered = plugin_manager.discover_plugins()

                    # Load plugins that have configurations
                    loaded_plugins = []
                    for plugin_data in discovered:
                        if plugin_data.name in plugin_configs:
                            try:
                                plugin_manager.load_plugin(plugin_data, plugin_configs[plugin_data.name])
                                loaded_plugins.append(plugin_data.name)
                            except Exception as e:
                                print(f"Worker: Failed to load plugin {plugin_data.name}: {e}")

                    # Send ready signal
                    result_queue.put({
                        'type': WorkerResponseType.READY.value,
                        'loaded_plugins': loaded_plugins
                    })

                elif request_type == WorkerRequestType.EXECUTE.value:
                    if not plugin_manager:
                        result_queue.put({
                            'type': WorkerResponseType.RESULT.value,
                            'job_id': request.get('job_id', 'unknown'),
                            'status': 'error',
                            'error': 'Plugin manager not initialized. Send init message first.'
                        })
                        continue

                    job_id = request['job_id']
                    plugin_name = request['plugin_name']

                    # Extract execution parameters (remove known keys)
                    exec_params = {k: v for k, v in request.items()
                                   if k not in ['type', 'job_id', 'plugin_name']}

                    try:
                        # Check if streaming is requested and supported
                        use_streaming = (
                            supports_streaming and
                            exec_params.get('stream', False) and
                            plugin_manager.check_streaming_support(plugin_name)
                        )

                        if use_streaming:
                            # Execute with streaming
                            for chunk in plugin_manager.execute_plugin_stream(plugin_name, **exec_params):
                                result_queue.put({
                                    'type': WorkerResponseType.STREAM_CHUNK.value,
                                    'job_id': job_id,
                                    'chunk': chunk,
                                    'is_final': False
                                })

                            # Send final chunk marker
                            result_queue.put({
                                'type': WorkerResponseType.STREAM_CHUNK.value,
                                'job_id': job_id,
                                'chunk': '',
                                'is_final': True
                            })

                            # Still send final result for completion tracking
                            result_queue.put({
                                'type': WorkerResponseType.RESULT.value,
                                'job_id': job_id,
                                'status': 'success',
                                'data': {'streamed': True}
                            })
                        else:
                            # Execute normally (non-streaming)
                            result = plugin_manager.execute_plugin(plugin_name, **exec_params)

                            # Adapt result if adapter provided
                            if result_adapter:
                                result_data = result_adapter(result)
                            elif isinstance(result, dict):
                                result_data = result
                            else:
                                # Assume result has text and metadata attributes
                                result_data = {
                                    'text': getattr(result, 'text', str(result)),
                                    'metadata': getattr(result, 'metadata', {})
                                }

                            # Update current state
                            current_plugin = plugin_name
                            if plugin_name in plugin_configs:
                                current_config = plugin_configs[plugin_name]

                            # Send success result
                            result_queue.put({
                                'type': WorkerResponseType.RESULT.value,
                                'job_id': job_id,
                                'status': 'success',
                                'data': result_data
                            })

                    except Exception as e:
                        # Send error result
                        result_queue.put({
                            'type': WorkerResponseType.RESULT.value,
                            'job_id': job_id,
                            'status': 'error',
                            'error': str(e)
                        })

                elif request_type == WorkerRequestType.UNLOAD.value:
                    if not plugin_manager:
                        response_queue.put({
                            'type': WorkerResponseType.RESPONSE.value,
                            'request_type': WorkerRequestType.UNLOAD.value,
                            'status': 'error',
                            'error': 'Plugin manager not initialized.'
                        })
                        continue

                    plugin_name = request.get('plugin_name')

                    try:
                        plugin_manager.unload_plugin(plugin_name)

                        # Clear state if this was the current plugin
                        if current_plugin == plugin_name:
                            current_plugin = None
                            current_config = None

                        response_queue.put({
                            'type': WorkerResponseType.RESPONSE.value,
                            'request_type': WorkerRequestType.UNLOAD.value,
                            'status': 'success',
                            'plugin_name': plugin_name
                        })
                    except Exception as e:
                        response_queue.put({
                            'type': WorkerResponseType.RESPONSE.value,
                            'request_type': WorkerRequestType.UNLOAD.value,
                            'status': 'error',
                            'error': str(e)
                        })

                elif request_type == WorkerRequestType.RELOAD.value:
                    if not plugin_manager:
                        print("[Worker] ✗ Plugin manager not initialized")
                        response_queue.put({
                            'type': WorkerResponseType.RESPONSE.value,
                            'request_type': WorkerRequestType.RELOAD.value,
                            'status': 'error',
                            'error': 'Plugin manager not initialized.'
                        })
                        continue

                    plugin_name = request.get('plugin_name')
                    new_config = request.get('config', {})

                    print(f"\n[Worker] Received reload request for: {plugin_name}")
                    print(f"[Worker] Previous plugin: {current_plugin}")

                    try:
                        # If switching plugins, unload the old one first to free GPU memory
                        if current_plugin and current_plugin != plugin_name:
                            print(f"[Worker] Plugin switch detected: {current_plugin} → {plugin_name}")
                            print(f"[Worker] Step 1: Unloading {current_plugin} to free GPU memory")
                            plugin_manager.reload_plugin(current_plugin)
                            print(f"[Worker] ✓ {current_plugin} unloaded successfully")

                        # Now reload with the new (or same) plugin
                        print(f"[Worker] Step 2: Calling plugin_manager.reload_plugin('{plugin_name}', config)")
                        plugin_manager.reload_plugin(plugin_name, new_config)
                        print(f"[Worker] ✓ Plugin reloaded successfully")

                        # Update state
                        current_plugin = plugin_name
                        current_config = new_config
                        plugin_configs[plugin_name] = new_config
                        print(f"[Worker] Updated worker state: current_plugin = {current_plugin}")

                        response_queue.put({
                            'type': WorkerResponseType.RESPONSE.value,
                            'request_type': WorkerRequestType.RELOAD.value,
                            'status': 'success',
                            'plugin_name': plugin_name
                        })
                        print(f"[Worker] Sent success response\n")
                    except Exception as e:
                        print(f"[Worker] ✗ Reload failed: {str(e)}")
                        response_queue.put({
                            'type': WorkerResponseType.RESPONSE.value,
                            'request_type': WorkerRequestType.RELOAD.value,
                            'status': 'error',
                            'error': str(e)
                        })

                elif request_type == WorkerRequestType.GET_STATE.value:
                    # Return current worker state
                    response_queue.put({
                        'type': WorkerResponseType.STATE.value,
                        'current_plugin': current_plugin,
                        'current_config': current_config,
                        'loaded_plugins': list(plugin_configs.keys())
                    })

            except queue.Empty:
                # No job available, continue waiting
                continue
            except KeyboardInterrupt:
                # Ctrl+C pressed, exit cleanly without traceback
                break
            except Exception as e:
                # Unexpected error in worker loop
                result_queue.put({
                    'type': WorkerResponseType.ERROR.value,
                    'error': f'Worker error: {str(e)}'
                })
                break

    except Exception as e:
        # Fatal error in worker initialization
        result_queue.put({
            'type': WorkerResponseType.ERROR.value,
            'error': f'Worker initialization failed: {str(e)}'
        })

### Communication Protocol

The worker communicates with the parent process using three queues:

**Request Queue** (Parent → Worker):
- `INIT`: Initialize plugin manager with configurations
- `EXECUTE`: Execute a job with specified plugin and parameters
- `RELOAD`: Reload a plugin with new configuration
- `UNLOAD`: Unload a plugin to free resources
- `GET_STATE`: Query current worker state
- `STOP`: Gracefully shutdown the worker

**Result Queue** (Worker → Parent):
- `READY`: Worker initialized successfully
- `RESULT`: Job execution completed (success or error)
- `STREAM_CHUNK`: Streaming output chunk
- `ERROR`: Fatal worker error

**Response Queue** (Worker → Parent):
- `RESPONSE`: Synchronous response to commands (reload, unload)
- `STATE`: Worker state information

### Worker Lifecycle

1. **Initialization**: Parent sends `INIT` message with plugin configurations
2. **Discovery**: Worker discovers available plugins
3. **Loading**: Worker loads configured plugins
4. **Execution Loop**: Worker processes `EXECUTE` requests
5. **Shutdown**: Parent sends `STOP` or terminates the process

### Model Reuse

The worker keeps models loaded in memory between jobs. When switching plugins, it unloads the old plugin first to free GPU memory before loading the new one.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()